## Database Ingestion

In this workshop we will create a SQL Server database, crawl the metadata with a Glue Crawler, and tarnsform the data into Parquet landing in our data lake for use with multiple analytics engines. The diagram below shows the workflow we will build in this notebook. This is a typical scenario where a customer wants to grab data from a relational database and make it available to their analytics groups.

**If connecting to an on-premises database please review this [blog](https://aws.amazon.com/blogs/big-data/how-to-access-and-analyze-on-premises-data-stores-using-aws-glue/) for requirements.** 

![SQL DB Crawl](../../docs/assets/images/sql-db-crawl.png)

In [ ]:
import boto3
import botocore
import json
import time
import os
import getpass

import project_path # path to helper methods
from lib import workshop
from pandas import read_sql

glue = boto3.client('glue')
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
cfn = boto3.client('cloudformation')
redshift_client = boto3.client('redshift')
ec2_client = boto3.client('ec2')
rds = boto3.client('rds')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

workshop_user = 'db'
database_name = 'sales' # AWS Glue Data Catalog Database Name
environment_name = 'workshop{0}'.format(workshop_user)

use_existing = True

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket(region, session, 'db-')
print(bucket)

### [Create VPC](https://docs.aws.amazon.com/vpc/index.html) 

We need a VPC for some of the resources in this workshop. You have the option to create a brand new VPC or use the VPC flaged as the default.

In [ ]:
if use_existing:
    vpc_filter = [{'Name':'isDefault', 'Values':['true']}]
    default_vpc = ec2_client.describe_vpcs(Filters=vpc_filter)
    vpc_id = default_vpc['Vpcs'][0]['VpcId']

    subnet_filter = [{'Name':'vpc-id', 'Values':[vpc_id]}]
    subnets = ec2_client.describe_subnets(Filters=subnet_filter)
    subnet1_id = subnets['Subnets'][0]['SubnetId']
    subnet2_id = subnets['Subnets'][1]['SubnetId']
else: 
    vpc, subnet1, subnet2 = workshop.create_and_configure_vpc()
    vpc_id = vpc.id
    subnet1_id = subnet1.id
    subnet2_id = subnet2.id

In [ ]:
print(vpc_id)
print(subnet1_id)
print(subnet2_id)

### Upload [CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/GettingStarted.html) template

In the interest of time we will leverage CloudFormation to launch our source SQL Server Database in RDS. The steps below would apply as well for databases supported with Glue that are not running in RDS.

In [ ]:
rds_file = 'sqlserver-rds.yaml'
file_path = os.path.join('cfn', rds_file)
session.resource('s3').Bucket(bucket).Object(file_path).upload_file(file_path)

### Enter the user name used for the SQL Server Database

**Must contain only alphanumeric characters.**

In [ ]:
admin_user = getpass.getpass()

### Enter password used for the SQL Server Database

**Must contain only alphanumeric characters with at least one capital letter and one number.**

In [ ]:
admin_password = getpass.getpass()

In [ ]:
import re

pattern = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)[a-zA-Z\d]{8,}$")
result = pattern.match(admin_password)
if result:
    print('Valid')
else:
    print('Invalid, Password must be 8 characters long alphanumeric only 1 Upper, 1 Lower')

### Execute CloudFormation Stack to create SQL Server RDS instance

In [ ]:
cfn_template = 'https://s3.amazonaws.com/{0}/cfn/{1}'.format(bucket, rds_file)
print(cfn_template)

rds_stack_name = 'SQLServerRDSStack'
response = cfn.create_stack(
    StackName=rds_stack_name,
    TemplateURL=cfn_template,
    Capabilities = ["CAPABILITY_NAMED_IAM"],
    Parameters=[
        {
            'ParameterKey': 'SqlServerInstanceName',
            'ParameterValue': environment_name
        },
        {
            'ParameterKey': 'DatabaseUsername',
            'ParameterValue': admin_user
        },
        {
            'ParameterKey': 'DatabasePassword',
            'ParameterValue': admin_password
        }
    ]
)

print(response)

### Check status of SQL Server RDS CloudFormaton stack

In [ ]:
response = cfn.describe_stacks(
    StackName=rds_stack_name
)

while response['Stacks'][0]['StackStatus'] != 'CREATE_COMPLETE':
    print('Not yet complete.')
    time.sleep(30)
    response = cfn.describe_stacks(
        StackName=rds_stack_name
    )

for output in response['Stacks'][0]['Outputs']:
    if (output['OutputKey'] == 'SQLDatabaseEndpoint'):
        rds_endpoint = output['OutputValue']
        print('RDS Endpoint: {0}'.format(rds_endpoint))
    if (output['OutputKey'] == 'SQLServerSecurityGroup'):
        rds_sg_id = output['OutputValue']
        print('RDS Security Group: {0}'.format(rds_sg_id))

### Create Sales DB

We will create a sample representation of a sales database containing customers, products, and relationships between them. To do so in this notebook we will be using the python library [`pymssql`](http://pymssql.org/en/stable/). In the first step we will create the `sales` database and then execute scripts located in this folder for the table structure and data populated in each table. Data for db from [www.dofactory.com](http://www.dofactory.com/sql/sample-database).

In [ ]:
!pip install pymssql

In [ ]:
import pymssql

conn = pymssql.connect(rds_endpoint, admin_user, admin_password)
conn.autocommit(True)

In [ ]:
cr=conn.cursor()
cr.execute('create database sales;')
conn.commit()

In [ ]:
def run_sql_file(filename, connection):
    '''
    The function takes a filename and a connection as input
    and will run the SQL query on the given connection  
    '''
    file = open(filename, 'r')
    sql = s = " ".join(file.readlines())
    cursor = connection.cursor()
    cursor.execute(sql)    
    connection.commit()

In [ ]:
run_sql_file('db-scripts/sample-model.sql', conn)

In [ ]:
run_sql_file('db-scripts/sample-data.sql', conn)

In [ ]:
run_sql_file('db-scripts/sample-view.sql', conn)

### Verify sample data loaded correctly

We will use pandas to visualize the data in the `Customer` table of the `sales` database.

In [ ]:
import pandas as pd

df = read_sql('SELECT top 10 * FROM sales.dbo.Customer ', con=conn)
df

In [ ]:
import pandas as pd

df = read_sql('SELECT top 10 * FROM sales.dbo.v_top_10_customers_by_order order by OrderCount desc', con=conn)
df

### Discover the data in your Data Lake

In this next section we will be using [AWS Glue](https://aws.amazon.com/glue/) to discover, catalog, and transform your data. Glue currently only supports `Python 2.7`, hence we'll write the script in `Python 2.7`.

### Permission setup for invoking AWS Glue from this Notebook
In order to enable this Notebook to run AWS Glue jobs, we need to add one additional permission to the default execution role of this notebook. We will be using SageMaker Python SDK to retrieve the default execution role and then you have to go to [IAM Dashboard](https://console.aws.amazon.com/iam/home) to edit the Role to add AWS Glue specific permission. 

### Finding out the current execution role of the Notebook
We are using SageMaker Python SDK to retrieve the current role for this Notebook which needs to be enhanced to support the functionality in AWS Glue.

In [ ]:
# Import SageMaker Python SDK to get the Session and execution_role
import sagemaker
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
role_name = role[role.rfind('/') + 1:]
print(role_name)


### Adding AWS Glue as an additional trusted entity to this role
This step is needed if you want to pass the execution role of this Notebook while calling Glue APIs as well without creating an additional **Role**. If you have not used AWS Glue before, then this step is mandatory. 

If you have used AWS Glue previously, then you should have an already existing role that can be used to invoke Glue APIs. In that case, you can pass that role while calling Glue (later in this notebook) and skip this next step.

On the IAM dashboard, please click on **Roles** on the left sidenav and search for this Role. Once the Role appears, click on the Role to go to its **Summary** page. Click on the **Trust relationships** tab on the **Summary** page to add AWS Glue as an additional trusted entity. 

Click on **Edit trust relationship** and replace the JSON with this JSON.
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
          "sagemaker.amazonaws.com",
          "glue.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
Once this is complete, click on **Update Trust Policy** and you are done.

![IAM Roles](../../docs/assets/images/iam_roles_hl.png)

In [ ]:
print("https://console.aws.amazon.com/iam/home?region={0}#/roles/{1}".format(region, role_name))

### Get RDS instance attributes

We need to get the `SecurityGroup`, `AvailabilityZone`, and `Subnet` the RDS instance is running in.

In [ ]:
response = rds.describe_db_instances(DBInstanceIdentifier=environment_name)

rds_sg_id = response['DBInstances'][0]['VpcSecurityGroups'][0]['VpcSecurityGroupId']
print(rds_sg_id)

rds_az = response['DBInstances'][0]['AvailabilityZone']
print(rds_az)

for sub in response['DBInstances'][0]['DBSubnetGroup']['Subnets']:
    if sub['SubnetAvailabilityZone']['Name'] == rds_az:
        rds_subnet = sub['SubnetIdentifier']
        print(rds_subnet)

### Create the [AWS Glue Catalog Database](https://docs.aws.amazon.com/glue/latest/dg/define-database.html)

When you define a table in the AWS Glue Data Catalog, you add it to a database. A database is used to organize tables in AWS Glue. You can organize your tables using a crawler or using the AWS Glue console. A table can be in only one database at a time.

There is a central Glue Catalog for each AWS account. When creating the database you will use your account id declared above as `account_id`

[glue.create_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_database)

In [ ]:
workshop.create_db(glue, account_id, database_name, 'Sales data in MS SQL Server database')

### [Create Glue SQL Connection](https://docs.aws.amazon.com/glue/latest/dg/populate-add-connection.html)

Connections are used by crawlers and jobs in AWS Glue to access certain types of data stores. We will create a connection to the `sales` MS SQL Server database we created above.

[glue.create_connection](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_connection)

In [ ]:
db_connection_name = 'MSSQLSalesConnection{0}'.format(workshop_user)

response = glue.create_connection(
    CatalogId=account_id,
    ConnectionInput={
        'Name': db_connection_name,
        'Description': 'MS SQL Server Sales Connection',
        'ConnectionType': 'JDBC',
        'MatchCriteria': [
            'string',
        ],
        'ConnectionProperties': {
            'JDBC_CONNECTION_URL': 'jdbc:sqlserver://{0};databaseName={1}'.format(rds_endpoint, database_name),
            'JDBC_ENFORCE_SSL': 'false',
            'PASSWORD': admin_password,
            'USERNAME': admin_user
        },
        'PhysicalConnectionRequirements': {
            'SubnetId': rds_subnet,
            'SecurityGroupIdList': [
                rds_sg_id,
            ],
            'AvailabilityZone': rds_az
        }
    }
)

print(response)

### Use a [Glue Crawler](https://docs.aws.amazon.com/glue/latest/dg/add-crawler.html) to Discover the transformed data

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

A crawler can crawl both file-based and table-based data stores. Crawlers can crawl the following data stores:

* Amazon Simple Storage Service (Amazon S3)
    * [Built-in Classifiers](https://docs.aws.amazon.com/glue/latest/dg/add-classifier.html#classifier-built-in)
    * [Custom Classifiers](https://docs.aws.amazon.com/glue/latest/dg/custom-classifier.html)
* Amazon Redshift
* Amazon Relational Database Service (Amazon RDS)
    * Amazon Aurora
    * MariaDB
    * Microsoft SQL Server
    * MySQL
    * Oracle
    * PostgreSQL
* Amazon DynamoDB
* Publicly accessible databases [Blog](https://aws.amazon.com/blogs/big-data/how-to-access-and-analyze-on-premises-data-stores-using-aws-glue/)
    * Aurora
    * MariaDB
    * SQL Server
    * MySQL
    * Oracle
    * PostgreSQL

[glue.create_crawler](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_crawler)

In [ ]:
crawler_name = 'MSSQL-Sales-Crawler'

response = glue.create_crawler(
    Name=crawler_name,
    Role=role,
    DatabaseName=database_name,
    Description='Crawler for SQL Server Sales Database',
    Targets={
        'JdbcTargets': [
            {
                'ConnectionName': db_connection_name,
                'Path': database_name
            },
        ]
    },
    TablePrefix='R_',
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
    }
)

### Start the Glue Crawler

You can use a crawler to populate the AWS Glue Data Catalog with tables. This is the primary method used by most AWS Glue users. You add a crawler within your Data Catalog to traverse your data stores. The output of the crawler consists of one or more metadata tables that are defined in your Data Catalog. Extract, transform, and load (ETL) jobs that you define in AWS Glue use these metadata tables as sources and targets.

[glue.start_crawler](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.start_crawler)

In [ ]:
response = glue.start_crawler(
    Name=crawler_name
)

print ("Crawler: https://{0}.console.aws.amazon.com/glue/home?region={0}#crawler:name={1}".format(region, crawler_name))


### Checking Glue crawler status

We will now monitor the crawler status waiting for it to get back into the `READY` state meaning the crawler completed it's crawl. You can also look at the [CloudWatch logs](https://docs.aws.amazon.com/glue/latest/dg/console-crawlers.html#console-crawlers-details) for the crawler for more details.

In [ ]:
crawler_status = glue.get_crawler(Name=crawler_name)['Crawler']['State']

while crawler_status not in ('READY'):
    crawler_status = glue.get_crawler(Name=crawler_name)['Crawler']['State']
    print(crawler_status)
    time.sleep(30)

### View Crawler Results

Now that we have crawled the raw data available, we want to look at the results of the crawl to see the tables that were created. You will click on the link `Tables in sales` to view the tables the crawler found. We prefixed the tables with an `R_` to signify they are raw tables from the source database. It will look like the image below:

![Sales Tables](../../docs/assets/images/sales-tables.png)

In [ ]:
print('https://{0}.console.aws.amazon.com/glue/home?region={0}#database:name={1}'.format(region, database_name))

### Create Parquet version and denormalize customer orders

From [Wikipedia](https://en.wikipedia.org/wiki/Apache_Parquet), "Apache Parquet is a free and open-source column-oriented data storage format of the Apache Hadoop ecosystem. It is similar to the other columnar-storage file formats available in Hadoop namely RCFile and ORC. It is compatible with most of the data processing frameworks in the Hadoop environment. It provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk." 

The key points in this code is how easy it is to get access to the AWS Glue Data Catalog leveraging the [Glue libraries](https://github.com/awslabs/aws-glue-libs). Some of the key concepts are below:

* [`glueContext.create_dynamic_frame.from_catalog`](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-glue-context.html#aws-glue-api-crawler-pyspark-extensions-glue-context-create_dynamic_frame_from_catalog) - Read table metadata from the Glue Data Catalog using Glue libs to load tables into the pyspark job.
* Writing back S3 [`glueContext.write_dynamic_frame.from_options`](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-glue-context.html#aws-glue-api-crawler-pyspark-extensions-glue-context-write_dynamic_frame_from_catalog) with options:
    * Convert data to different format `format="parquet"`. This format is [columnar](https://docs.aws.amazon.com/athena/latest/ug/columnar-storage.html) and provides [Snappy](https://en.wikipedia.org/wiki/Snappy_(compression)) compression by default.
    * Partition the data. The script is using `"partitionKeys": ["customerid"]` to partition the data under the S3 prefix to look like the following `s3://datalake..../sales_parquet/customer=[n]`.

**This script is sacrificing best practices in parquet size to demonstrate features of the Glue libraries.**

You can find more best practices for Glue and Athena [here](https://docs.aws.amazon.com/athena/latest/ug/glue-best-practices.html).

In [ ]:
%%writefile sales_parquet_etl.py
import sys
import os
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_OUTPUT_BUCKET', 'S3_OUTPUT_KEY_PREFIX', 'DATABASE_NAME', 'REGION'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

datasource0 = glueContext.create_dynamic_frame.from_catalog(database=args['DATABASE_NAME'], table_name='r_sales_dbo_customer', transformation_ctx = "datasource0")

datasource1 = glueContext.create_dynamic_frame.from_catalog(database=args['DATABASE_NAME'], table_name='r_sales_dbo_order', transformation_ctx = "datasource1")

datasource2 = RenameField.apply(datasource1, "id", "orderid")

datasource3 = datasource2.join( ["customer_id"],["id"], datasource0, transformation_ctx = "join")

applymapping1 = ApplyMapping.apply(frame = datasource3, mappings = [("firstname", "string", "firstname", "string"), \
                                                                    ("phone", "string", "phone", "string"), ("country", "string", "country", "string"), \
                                                                    ("id", "int", "id", "int"), ("lastname", "string", "lastname", "string"), \
                                                                    ("city", "string", "city", "string"), ("ordernumber", "string", "ordernumber", "string"), \
                                                                    ("orderid", "int", "orderid", "int"), ("customerid", "int", "customerid", "int"), \
                                                                    ("totalamount", "decimal(12,2)", "totalamount", "decimal(12,2)"), ("orderdate", "timestamp", "orderdate", "timestamp")], transformation_ctx = "applymapping1")

selectfields1 = SelectFields.apply(frame = applymapping1, paths = ["ordernumber", "city", "firstname", "lastname", "totalamount", "phone", "customerid", "orderdate", "country"], transformation_ctx = "selectfields1")

parquet_output_path = 's3://' + os.path.join(args['S3_OUTPUT_BUCKET'], args['S3_OUTPUT_KEY_PREFIX'])

datasink3 = glueContext.write_dynamic_frame.from_options(frame = selectfields1, connection_type = "s3", connection_options = {"path": parquet_output_path, "partitionKeys": ["customerid"]}, format = "parquet", transformation_ctx = "datasink4")
job.commit()

### Upload the ETL script to S3
We will be uploading the `sales_parquet_etl` script to S3 so Glue can use it to run the PySpark job. You can replace it with your own script if needed. If your code has multiple files, you need to zip those files and upload to S3 instead of uploading a single file like it's being done here.

In [ ]:
script_location = sess.upload_data(path='sales_parquet_etl.py', bucket=bucket, key_prefix='codes')

In [ ]:
# Output location of the data.
s3_output_key_prefix = 'datalake/sales_parquet/'

### [Authoring jobs with AWS Glue](https://docs.aws.amazon.com/glue/latest/dg/author-job.html)

Next we'll be creating Glue client via Boto so that we can invoke the `create_job` API of Glue. `create_job` API will create a job definition which can be used to execute your jobs in Glue. The job definition created here is mutable. While creating the job, we are also passing the code location as well as the dependencies location to Glue.

`AllocatedCapacity` parameter controls the hardware resources that Glue will use to execute this job. It is measures in units of `DPU`. For more information on `DPU`, please see [here](https://docs.aws.amazon.com/glue/latest/dg/add-job.html).

[glue.create_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_job)

In [ ]:
from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

job_name = 'sales-parquet-' + timestamp_prefix
response = glue.create_job(
    Name=job_name,
    Description='PySpark job to convert sales SQL Server tables data to parquet',
    Role=role, # you can pass your existing AWS Glue role here if you have used Glue before
    ExecutionProperty={
        'MaxConcurrentRuns': 1
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': script_location
    },
    DefaultArguments={
        '--job-language': 'python',
        '--job-bookmark-option': 'job-bookmark-disable'
    },
    AllocatedCapacity=5,
    Timeout=60,
)
glue_job_name = response['Name']
print(glue_job_name)

The aforementioned job will be executed now by calling `start_job_run` API. This API creates an immutable run/execution corresponding to the job definition created above. We will require the `job_run_id` for the particular job execution to check for status. We'll pass the data and model locations as part of the job execution parameters.

[glue.start_job_run](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.start_job_run)

In [ ]:
job_run_id = glue.start_job_run(JobName=job_name,
                                       Arguments = {
                                        '--S3_OUTPUT_BUCKET': bucket,
                                        '--S3_OUTPUT_KEY_PREFIX': s3_output_key_prefix,
                                        '--DATABASE_NAME': database_name,
                                        '--REGION': region
                                       })['JobRunId']
print(job_run_id)

### Checking Glue Job Status

Now we will check for the job status to see if it has `SUCCEEDED`, `FAILED` or `STOPPED`. Once the job is succeeded, we have the transformed data into S3 in Parquet format which we will use to query with Athena and visualize with QuickSight. If the job fails, you can go to AWS Glue console, click on **Jobs** tab on the left, and from the page, click on this particular job and you will be able to find the CloudWatch logs (the link under **Logs**) link for these jobs which can help you to see what exactly went wrong in the job execution.

[glue.get_job_run](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.get_job_run)

In [ ]:
job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
while job_run_status not in ('FAILED', 'SUCCEEDED', 'STOPPED'):
    job_run_status = glue.get_job_run(JobName=job_name,RunId=job_run_id)['JobRun']['JobRunState']
    print (job_run_status)
    time.sleep(60)
print(job_run_status)

### Create Crawler to populate Parquet formated table in Glue Data Catalog

We will create another crawler for the curated dataset we created converting the CSV files into Parquet formatted data.

In [ ]:
parq_crawler_name = 'Sales-Curated-Crawler'
parq_crawler_path = 's3://{0}/{1}'.format(bucket, s3_output_key_prefix)

response = glue.create_crawler(
    Name=parq_crawler_name,
    Role=role,
    DatabaseName=database_name,
    Description='Crawler for the Parquet transformed sales data',
    Targets={
        'S3Targets': [
            {
                'Path': parq_crawler_path
            }
        ]
    },
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
    }
)

### Start Crawler 

Much like we did with the raw data crcawler we will start the curated crawler pointing to the new data set created from the Glue job.

In [ ]:
response = glue.start_crawler(
    Name=parq_crawler_name
)

print ("Crawler: https://{0}.console.aws.amazon.com/glue/home?region={0}#crawler:name={1}".format(region, parq_crawler_name))

### Monitor the status of the Parquet crawler

In [ ]:
crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']

while crawler_status not in ('READY'):
    crawler_status = glue.get_crawler(Name=parq_crawler_name)['Crawler']['State']
    print(crawler_status)
    time.sleep(30)

In [ ]:
print('https://{0}.console.aws.amazon.com/glue/home?region={0}#database:name={1}'.format(region, database_name))

### [Query the Data Lake with Athena](https://aws.amazon.com/athena/)

For the self-serve end users that need the ability to create ad-hoc queries against the data Athena is a great choice the utilizes Presto and ANSI SQL to query a number of file formats on S3.

To query the tables created by the crawler we will be installing a python library for querying the data in the Glue Data Catalog with Athena. For more information jump to [PyAthena](https://pypi.org/project/PyAthena/). You can also use the AWS console by browsing to the Athena service and run queries through the browser. Alternatively, you can also use the [JDBC/ODBC](https://docs.aws.amazon.com/athena/latest/ug/athena-bi-tools-jdbc-odbc.html) drivers available.

In [ ]:
!pip install PyAthena

### Simple Select Query

In this first query we will create a simple query to show the ability of Athena to query the raw CSV data.

In [ ]:
%%time
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region, s3_staging_dir='s3://'+bucket+'/athena/temp').cursor()
cursor.execute('select customerid, count(ordernumber) as order_cnt from ' + database_name + '.sales_parquet group by customerid')

df = as_pandas(cursor)

In [ ]:
df.head(5)

Congratulations, you have completed loading data into your data lake from a SQL Server database. This completes the workshop you can now run the cleanup scripts to remove all resources created in this account.

## Clean Up

In [ ]:
response = cfn.delete_stack(StackName=rds_stack_name)

In [ ]:
response = glue.delete_crawler(Name=crawler_name)

In [ ]:
response = glue.delete_crawler(Name=parq_crawler_name)

In [ ]:
response = glue.delete_job(JobName=glue_job_name)

In [ ]:
response = glue.delete_database(
    CatalogId = account_id,
    Name = database_name
)

In [ ]:
response = glue.delete_connection(
    CatalogId=account_id,
    ConnectionName=db_connection_name
)

In [ ]:
waiter = cfn.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=rds_stack_name
)

print('The wait is over for {0}'.format(rds_stack_name))

In [ ]:
!aws s3 rb s3://$bucket --force  

In [ ]:
if not use_existing:
    workshop.vpc_cleanup(vpc_id)